In [0]:
FITS_PATH = '/scratch/aszalay1/dobos/pfs/gapipe/out/dSph_draco_2025-03_November2025/pfsStarCatalog/10092/051-0x310d8290f6dc2641/pfsStarCatalog-10092-051-0x310d8290f6dc2641.fits'
TITLE = "Draco 2025-03"
OBSTIME = "2025-03-30T14:03:47.828450"
V_LOS = -292

# FITS_PATH = '/scratch/aszalay1/dobos/pfs/gapipe/out/S25A_November2025/pfsStarCatalog/10092/024-0x17ca7c3d6eb520d8/pfsStarCatalog-10092-024-0x17ca7c3d6eb520d8.fits'
# TITLE = "Draco 2025-06"
# OBSTIME = "2025-07-01T07:18:03"
# V_LOS = -292

# FITS_PATH = '/scratch/aszalay1/dobos/pfs/gapipe/out/dSph_ursaminor_2025-03_November2025/pfsStarCatalog/10092/073-0x6fd72eb843599f0f/pfsStarCatalog-10092-073-0x6fd72eb843599f0f.fits'
# TITLE = "Ursa Minor 2025-03"
# OBSTIME = "2025-03-30T14:03:47"
# V_LOS = -247

# FITS_PATH = '/scratch/aszalay1/dobos/pfs/gapipe/out/S25A_November2025/pfsStarCatalog/10092/023-0x4955f4338c4bfcc8/pfsStarCatalog-10092-023-0x4955f4338c4bfcc8.fits'
# TITLE = "Ursa Minor 2025-06"
# OBSTIME = "2025-07-01T07:18:03"
# V_LOS = -247

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pfs.datamodel import PfsStarCatalog, TargetType, FiberStatus, TempFitFlag

In [0]:
catalog = PfsStarCatalog.readFits(FITS_PATH)

In [0]:
catalog.catalog.columns.keys()

In [0]:
nodata_mask = np.char.find(catalog.catalog.status, 'NODATA') != -1

print('Total stars in catalog', len(catalog.catalog))
print('Stars with NODATA status', np.sum(nodata_mask))
print('Stars with valid data', np.sum(~nodata_mask))
print('Stars observed with one fiber', np.sum((catalog.catalog.fiberId != -1) & ~nodata_mask))
print('Stars observed with multiple fibers', np.sum((catalog.catalog.fiberId == -1) & ~nodata_mask))

In [0]:
# Multiply observed stars
catalog.catalog.objId[(catalog.catalog.fiberId == -1) & ~nodata_mask]

In [0]:
print('OK', np.sum(catalog.catalog.status == ''))

for t in TempFitFlag:
    print(t.name, np.sum(np.char.find(catalog.catalog.status, t.name) != -1))

In [0]:
for pri in sorted(np.unique(catalog.catalog.targetPriority)):
    print(f'Target priority {pri}: {np.sum(catalog.catalog.targetPriority == pri)} stars')

In [0]:
# Histogram of successful visits
hist = np.bincount(catalog.catalog.nVisit[catalog.catalog.nVisit > 0])
bins = np.arange(len(hist))
plt.bar(bins, hist)
plt.xlabel('Number of visits')
plt.ylabel('Number of stars')

In [0]:
ra, dec = catalog.catalog.columns['ra'].mean(), catalog.catalog.columns['dec'].mean()
ra, dec

In [0]:
from astropy.coordinates import EarthLocation, AltAz, EarthLocation
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord

# Create observer location (PFS is at Subaru, Hawaii)
subaru = EarthLocation(lat=19.8255*u.deg, lon=-155.4735*u.deg, height=4139*u.m)

# Create sky coordinate and time object
coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
time = Time(OBSTIME, scale='utc')

# Calculate heliocentric correction
helio_corr = coord.radial_velocity_correction(obstime=time, location=subaru, kind='heliocentric').to(u.km/u.s)
print(f"Heliocentric correction: {helio_corr.value}")

# Calculate the barycentric correction
bary_corr = coord.radial_velocity_correction(obstime=time, location=subaru, kind='barycentric').to(u.km/u.s)
print(f"Barycentric correction: {bary_corr.value}")

In [0]:
np.sum(catalog.catalog.T_effStatus != '')

In [0]:
mask = (catalog.catalog.T_effStatus == '') & (catalog.catalog.log_gStatus == '')

plt.plot(catalog.catalog.T_eff[~mask], catalog.catalog.log_g[~mask], '.', ms=1, c='black')
plt.plot(catalog.catalog.T_eff[mask], catalog.catalog.log_g[mask], '.', ms=1, c='red')
plt.xlabel('T_eff [K]')
plt.ylabel('log_g')
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.title(TITLE)

In [0]:
l = plt.scatter(
    catalog.catalog.T_eff, catalog.catalog.log_g, 
    c=catalog.catalog.targetPriority, cmap='tab10',
    marker='s', s=2, edgecolor='none')
plt.xlabel('T_eff [K]')
plt.ylabel('log_g')
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.title(TITLE)
plt.colorbar(l, label='Target Priority')

In [0]:
mask = (catalog.catalog.T_effStatus == '') & (catalog.catalog.log_gStatus == '')

plt.plot(catalog.catalog.v_los[~mask], catalog.catalog.log_g[~mask], '.', ms=1, c='black')
plt.plot(catalog.catalog.v_los[mask], catalog.catalog.log_g[mask], '.', ms=1, c='red')
plt.axvline(V_LOS - helio_corr.value, color='blue', ls='--')
plt.axvline(V_LOS, color='blue', ls='-')
plt.xlabel('v_los [km/s]')
plt.ylabel('log_g')
plt.gca().invert_yaxis()
plt.title(TITLE)

In [0]:
l = plt.scatter(
    catalog.catalog.v_los, catalog.catalog.log_g,
    c=catalog.catalog.targetPriority, cmap='tab10',
    marker='s', s=2, edgecolor='none')
# plt.axvline(V_LOS - helio_corr.value, color='blue', ls='--')
# plt.axvline(V_LOS, color='blue', ls='-')
plt.xlabel('v_los [km/s]')
plt.ylabel('log_g')
plt.gca().invert_yaxis()
plt.title(TITLE)
plt.colorbar(l, label='Target Priority')

In [0]:
mask = (catalog.catalog.T_effStatus == '') & (catalog.catalog.log_gStatus == '')

bins = 100

hist, bins = np.histogram(catalog.catalog.v_los[~mask], bins=bins)
plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, color='black')

# hist, bins = np.histogram(catalog.catalog.v_los[mask], bins=bins)
# plt.step(bins[:-1], hist, where='post', color='red')

hist, bins = np.histogram(catalog.catalog.v_los, bins=bins)
plt.plot(0.5 * (bins[:-1] + bins[1:]), hist, color='red')

plt.axvline(V_LOS - helio_corr.value, color='blue', ls='--')
plt.axvline(V_LOS, color='blue', ls='-')

plt.xlabel('v_los [km/s]')
plt.ylabel('Number of stars')
plt.title(TITLE)

In [0]:
# Stacked histogram of v_los for different target priorities
bins = np.linspace(-400, 400, 50)
old_hist = np.zeros(len(bins) - 1)

for pri in sorted(np.unique(catalog.catalog.targetPriority)):
    mask = (catalog.catalog.targetPriority == pri) & (catalog.catalog.T_effStatus == '') & (catalog.catalog.log_gStatus == '')
    hist, bins = np.histogram(catalog.catalog.v_los[mask], bins=bins)
    plt.fill_between(0.5 * (bins[:-1] + bins[1:]), old_hist, old_hist + hist, alpha=0.5, label=f'Priority {pri}')
    old_hist += hist
plt.xlabel('v_los [km/s]')
plt.ylabel('Number of Stars')
plt.title('Stacked Histogram of v_los for Different Target Priorities')
plt.legend()

In [0]:
mask = (catalog.catalog.T_effStatus == '') & (catalog.catalog.log_gStatus == '')

plt.plot(catalog.catalog.v_los[~mask], catalog.catalog.M_H[~mask], '.', ms=1, c='black')
plt.plot(catalog.catalog.v_los[mask], catalog.catalog.M_H[mask], '.', ms=1, c='red')
plt.axvline(V_LOS - helio_corr.value, color='blue', ls='--')
plt.axvline(V_LOS, color='blue', ls='-')
plt.xlabel('v_los [km/s]')
plt.ylabel('[M/H]')
# plt.gca().invert_yaxis()
plt.title(TITLE)

In [0]:
l = plt.scatter(
    catalog.catalog.v_los, catalog.catalog.M_H,
    c=catalog.catalog.targetPriority, cmap='tab10',
    marker='s', s=2, edgecolor='none')

# plt.axvline(V_LOS - helio_corr.value, color='blue', ls='--')
# plt.axvline(V_LOS, color='blue', ls='-')
plt.xlabel('v_los [km/s]')
plt.ylabel('[M/H]')
# plt.gca().invert_yaxis()
plt.title(TITLE)
plt.colorbar(l, label='Target Priority')

In [0]:
mask = np.isfinite(catalog.catalog.snr_m)

hist, bins = np.histogram(catalog.catalog.snr_m[mask], bins=50)

plt.step(bins[:-1], hist, where='post', color='black')

plt.xlabel('SNR_M')
plt.ylabel('Number of stars')
plt.title(TITLE)

In [0]:
mask = (catalog.catalog.v_losStatus == '')

plt.plot(catalog.catalog.snr_m[~mask], catalog.catalog.v_losErr[~mask], '.', ms=1)
plt.plot(catalog.catalog.snr_m[mask], catalog.catalog.v_losErr[mask], '.', ms=1)
plt.xlim(0, 5)
plt.ylim(0, 50)
plt.xlabel('SNR_M')
plt.ylabel('v_losErr [km/s]')

In [0]:
# Histogram of v_los error
plt.hist(catalog.catalog.v_losErr, bins=50, range=(0, 20))
plt.hist(catalog.catalog.v_losErr[mask], bins=50, range=(0, 20))
plt.xlabel('v_losErr [km/s]')
plt.ylabel('Number of stars')
plt.title(TITLE)